In [56]:
import util as ut

In [57]:
mdp = ut.get_file("2_mdps.pickle")
print(f"File contains info for {len(mdp)} MDPs.")

File contains info for 2 MDPs.


In [58]:
flight_index = 0
costs = mdp[flight_index][1][1]
actions = mdp[flight_index][1][3]
probabilities = mdp[flight_index][1][5]

In [59]:
# time horizon
T = len(costs)
print(f"Time horizon: {T}")

Time horizon: 24


Find maximum number of neighbors to determine size of action space:

In [60]:
max_num_neighbors = 0
for t in range(T):
    list_lengths = [len(lst) for lst in actions[t].values()]
    max_num_neighbors = max(max_num_neighbors, max(list_lengths))
print(f"Max number of neighbors: {max_num_neighbors}")
numActions = max_num_neighbors

Max number of neighbors: 75


Compute set of states (all visible neighbors for all timesteps, with duplicates removed). Then map set (with missing values) to range 0 to n-1, in order to reduce matrix size

In [61]:
# Find all visible states for all time steps
states = set(key for state_at_time in actions for key in state_at_time.keys())
s2i = {value: index for index, value in enumerate(states)}
i2s = {index: value for index, value in enumerate(states)}
i2s[s2i[13558]] == 13558 # sanity check
numStates = len(states)
print(f"Number of states: {numStates}")

Number of states: 10867


Data set checks

In [62]:
# Check data set consistency: every (state, action) pair has #transitions == #neighbors == #actions
for t in range(T):
    visible_states = actions[t].keys()
    for state in visible_states:
        #print(f"({t}, {state}):      {actions[t][state]}")
        n = len(actions[t][state])
        for action in range(n):
            assert len(probabilities[t][(state, action)]) == n

In [63]:
# Check that all probabilities sum to 1
for t in range(T):
    visible_states = actions[t].keys()
    for state in visible_states:
        n = len(actions[t][state])
        for action in range(n):
            assert abs(sum(probabilities[t][(state, action)]) - 1) < 1e-6

In [64]:
# row of transition matrix for state 13558 action 0, time 0
print(probabilities[0][(13558, 0)])

# neighbors of state 13558
print(actions[0][13558])

# example insertion into non-flattened transition probability tensor
for i in range(len(actions[0][13558])):
    if probabilities[0][(13558, 0)][i] > 0: # don't insert 0 probabilities
        print(f"insert {probabilities[0][(13558, 0)][i]} at index {s2i[actions[0][13558][i]]}")

[0.95, 0, 0, 0.050000000000000044, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[11522, 11524, 15241, 13558, 11523, 15240, 10, 14, 13547, 13546, 13548, 12, 15242]
insert 0.95 at index 8362
insert 0.050000000000000044 at index 9704


In [65]:
# Mappping for flattened transition probability tensor (uses state index instead of state id!)
def global_row_P(time, state, action):
    return time * numStates * numActions + state * numActions + action

def inv_global_row_P(index):
    time = index // (numStates * numActions)
    state = (index % (numStates * numActions)) // numActions
    action = index % numActions
    return time, state, action

# Col mappings for flattened transition probability tensor (also row mapping for stage cost matrix)
def global_col_P(time, state):
    return time * numStates + state

def inv_global_col_P(index):
    time = index // numStates
    state = index % numStates
    return time, state

Create scipy transition probability matrix

In [66]:
import numpy as np
import scipy.sparse as sp

# Prepare lists for row indices, column indices and values
rows = []
cols = []
values = []

# fill in values
for t in range(T):
    visible_states = actions[t].keys()
    for s in visible_states:
        neighbors_ts = actions[t][s]
        numActions_ts = len(neighbors_ts)
        for a in range(numActions_ts):
            row = global_row_P(t, s2i[s], a)
            probs_tsa = probabilities[t][(s, a)]
            # filter out 0 probabilities
            vals = [p for p, c in zip(probs_tsa, neighbors_ts) if p > 0]
            col_indices = [global_col_P(t, s2i[c]) for p, c in zip(probs_tsa, neighbors_ts) if p > 0]

            # We add the row index to 'rows' as many times as there are values for that row
            rows.extend([row] * len(vals))
            cols.extend(col_indices)
            values.extend(vals)

# Create COO matrix
P_coo = sp.coo_matrix((values, (rows, cols)), dtype=np.float64, shape=(T * numStates * numActions, T * numStates))
print(transitionProbabilityTensor.shape)
# Convert to CSR format
transitionProbabilityTensor = P_coo.tocsr()
print(transitionProbabilityTensor.shape)


(19560600, 260808)
(19560600, 260808)
